In [1]:
import pandas as pd

**Import and prepare the data**

In [2]:
def prepare_data(coin):
    df = pd.read_csv(f'../data/coin_{coin}.csv', index_col=0).reset_index()
    df.Date = pd.to_datetime(df.Date)
    df['dayofweek'] = df.Date.dt.dayofweek
    df['dayofmonth'] = df.Date.dt.day
    df['year_month'] = df.Date.dt.strftime("%Y-%m")
    df['avg_daily_price']= (df.High + df.Low) / 2
    return df

In [3]:
def calculate_profit_dca(df, frequency, purchase_year_month = '2018-01',\
                        purchase_day_of_week = 0, purchase_day_of_month = [0]):
    purchase_usd = 10
    if frequency == 'monthly':
        freq_cond = (df.dayofmonth.isin(purchase_day_of_month))
        if(31 in purchase_day_of_month):
            # Take last day of months with 30 days
            freq_cond = freq_cond | (df.Date.dt.month.isin([11,4,6,9]) & (df.dayofmonth == 30))
        if((31 in purchase_day_of_month) or (30 in purchase_day_of_month)):
            # Take last day of February (29th in case of leap year)
            freq_cond = freq_cond |\
                        (df.Date.dt.year.isin([2008, 2012, 2016, 2020, 2024, 2028, 2032, 2036, 2040, 2044]) &\
                         df.Date.dt.month.isin([2]) & (df.dayofmonth == 29))
        if((31 in purchase_day_of_month) or (30 in purchase_day_of_month) or (29 in purchase_day_of_month)):     
            # In case 29 is the day, use 28 for February in non leap year
            freq_cond = freq_cond |\
                        ((~df.Date.dt.year.isin([2008, 2012, 2016, 2020, 2024, 2028, 2032, 2036, 2040, 2044])) &\
                         df.Date.dt.month.isin([2]) & (df.dayofmonth == 28))          
    if frequency == 'weekly':
        freq_cond = (df.dayofweek == purchase_day_of_week)

    purchase = df[(df.year_month >= purchase_year_month) & freq_cond]
    purchased_amount = (purchase_usd/purchase.High).sum()
    investment_amount = purchase_usd * len(purchase)
    investment_value = df[-1:].High * purchased_amount
    return (investment_value / investment_amount)

In [4]:
dfs = {}
coins = ['Bitcoin', 'Ethereum']

**Monthly**

In [5]:
days= []
monthly_profit = {}
for i in range(1,32):
    days.append(i)

for coin in coins:
    df = prepare_data(coin)
    monthly_profit[coin] = []
    print(f"Calculating profit for coin {coin}...")
    for i in range(1,32):    
        monthly_profit[coin].append(calculate_profit_dca(df, 'monthly', purchase_day_of_month=[i]).iloc[0])

Calculating profit for coin Bitcoin...
Calculating profit for coin Ethereum...


In [6]:
monthly_profit['days']=days
monthly_df = pd.DataFrame.from_dict(monthly_profit).set_index('days')
dfs['monthly'] = monthly_df
monthly_df.head()

,Bitcoin,Ethereum
days,,
1,4.043401,8.670210
2,3.992209,8.522013
3,3.956897,8.473842
4,3.987841,8.543198
5,3.992413,8.512712


**Weekly**

In [7]:
days= []
weekly_profit = {}
for i in range(0,7):
    days.append(i)

for coin in coins:
    df = prepare_data(coin)
    weekly_profit[coin] = []
    print(f"Calculating profit for coin {coin}...")
    for i in range(0,7):    
        weekly_profit[coin].append(calculate_profit_dca(df, 'weekly', purchase_day_of_week=i).iloc[0])

Calculating profit for coin Bitcoin...
Calculating profit for coin Ethereum...


In [8]:
weekly_profit['days']=days
weekly_df = pd.DataFrame.from_dict(weekly_profit).set_index('days')
dfs['weekly'] = weekly_df
weekly_df.head()

,Bitcoin,Ethereum
days,,
0,4.049776,8.685145
1,4.055809,8.732725
2,4.061459,8.746297
3,4.059354,8.754766
4,4.087846,8.822309


**Twice a month**

In [9]:
days= []
twice_a_month_profit = {}
for i in range(1,32):
    for j in range(i+1,32):
        days.append(f"{i}-{j}")

for coin in coins:
    df = prepare_data(coin)
    twice_a_month_profit[coin] = []
    print(f"Calculating profit for coin {coin}...")
    for i in range(1,32):
        for j in range(i+1,32):    
            twice_a_month_profit[coin].append(calculate_profit_dca(df, 'monthly', purchase_day_of_month=[i,j]).iloc[0])

Calculating profit for coin Bitcoin...
Calculating profit for coin Ethereum...


In [10]:
twice_a_month_profit['days']=days
twice_a_month_df = pd.DataFrame.from_dict(twice_a_month_profit).set_index('days')
dfs['twice_a_month'] = twice_a_month_df
twice_a_month_df.head()

,Bitcoin,Ethereum
days,,
1-2,4.017805,8.596112
1-3,4.000149,8.572026
1-4,4.015621,8.606704
1-5,4.017907,8.591461
1-6,4.013217,8.564750


In [11]:
coin_describe_values = {}
for coin in coins:
    describe_values = {}
    for index, df in dfs.items():
        describe_values[index] = df.describe()[coin].values
    coin_describe_values[coin] = describe_values

In [12]:
coin_describe_dfs = {}
for coin in coins:
    coin_describe_dfs[coin] = pd.DataFrame\
                              .from_dict(coin_describe_values[coin], orient='index', columns=df.describe().index)

In [13]:
coin_describe_dfs['Bitcoin'].transpose()

,monthly,weekly,twice_a_month
count,31.000000,7.000000,465.000000
mean,4.067292,4.067256,4.067089
std,0.052084,0.014194,0.035807
min,3.956897,4.049776,3.969964
25%,4.041394,4.057582,4.044083
50%,4.064771,4.061459,4.067624
75%,4.104478,4.078274,4.093047
max,4.149641,4.087846,4.147238


In [14]:
coin_describe_dfs['Ethereum'].transpose()

,monthly,weekly,twice_a_month
count,31.000000,7.000000,465.000000
mean,8.742306,8.752661,8.742688
std,0.151522,0.042145,0.105216
min,8.459290,8.685145,8.466566
25%,8.634049,8.739511,8.670926
50%,8.765556,8.746926,8.749390
75%,8.881335,8.767613,8.823540
max,8.944014,8.822309,8.946835


* The montly and twice-a-month DCA have provided the highest profits, when we look at the maxmium obtained.
* The monthly strategy presents however the lowest minumum profits. Indeed its standard deviation is the highest
* The weekly approach has provided more stable profits, regardless of the day picked for investing, but presents a lower maximum profits compared to the other approaches
* In general it seems that the higher the frequency of the DCA, the less influent the day on which the investment is made and the more stable profits. The lower the frequency, the higher the maximum possible profits, but also higher standard deviation and the risk of obtaining lower profits if not picking the right day to invest

We conclude that the best approach for those aiming at maximising the profit should use a twice-a-month approach. It has shown to provide similar maximum profits, compared to the monthly frequency, but with a lower standard deviation, which means less sensible to variations given the day chose to invest on.
For a more stable profit, weekly seems to provide more constant profits, regardless of the days on which the investment is made. Nevertheless, the maximum profit obtained with the weekly frequency is lower than the other approaches.